In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/06 20:10:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.version

'3.4.4'

In [5]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-06 20:10:24--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.155.128.6, 18.155.128.46, 18.155.128.187, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.155.128.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet.2’

yellow_tripdata_202 100%[===================>]  61.36M   129MB/s    in 0.5s    

2025-03-06 20:10:24 (129 MB/s) - ‘yellow_tripdata_2024-10.parquet.2’ saved [64346071/64346071]



In [6]:
df = spark.read \
    .option("header", "true") \
    .parquet('yellow_tripdata_2024-10.parquet')

In [7]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [8]:
df \
    .repartition(4) \
    .write.parquet('hw', mode='overwrite')

In [9]:
!ls -lh hw/

total 90M
-rw-r--r-- 1 polina polina   0 Mar  6 20:10 _SUCCESS
-rw-r--r-- 1 polina polina 23M Mar  6 20:10 part-00000-d943919e-6494-4e2b-b75a-300f4327f039-c000.snappy.parquet
-rw-r--r-- 1 polina polina 23M Mar  6 20:10 part-00001-d943919e-6494-4e2b-b75a-300f4327f039-c000.snappy.parquet
-rw-r--r-- 1 polina polina 23M Mar  6 20:10 part-00002-d943919e-6494-4e2b-b75a-300f4327f039-c000.snappy.parquet
-rw-r--r-- 1 polina polina 23M Mar  6 20:10 part-00003-d943919e-6494-4e2b-b75a-300f4327f039-c000.snappy.parquet


In [10]:
df.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [11]:
df.registerTempTable('yellow_2024_10')

/home/polina/spark/spark-3.4.4-bin-hadoop3/python/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [12]:
# How many taxi trips were there on the 15th of October?
# Consider only trips that started on the 15th of October.

df_result = spark.sql("""
SELECT 
    count(*)
FROM 
    yellow_2024_10
WHERE
    tpep_pickup_datetime >= '2024-10-15 00:00:00'
    AND
    tpep_pickup_datetime < '2024-10-16 00:00:00'
;
""").show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



In [15]:
# What is the length of the longest trip in the dataset in hours?

df_longest_trip = spark.sql("""
SELECT 
    MAX(TIMESTAMPDIFF(hour,tpep_pickup_datetime,tpep_dropoff_datetime)) AS longest_trip
FROM 
    yellow_2024_10
;
""").show()

+------------+
|longest_trip|
+------------+
|         162|
+------------+



In [16]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-06 20:12:08--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.155.128.46, 18.155.128.6, 18.155.128.187, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.155.128.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv.1’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-03-06 20:12:08 (180 MB/s) - ‘taxi_zone_lookup.csv.1’ saved [12331/12331]



In [19]:
df_zones = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

In [20]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [21]:
# Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?

df_with_zones = df.join(df_zones, df.PULocationID == df_zones.LocationID)

In [24]:
df_with_zones.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee',
 'LocationID',
 'Borough',
 'Zone',
 'service_zone']

In [26]:
df_with_zones.registerTempTable('with_zones')

In [30]:
df_least_frequent = spark.sql("""
SELECT 
    PULocationID,
    Zone,
    count(*)
FROM 
    with_zones
GROUP BY
    1,2
ORDER BY
    count(*) asc
LIMIT
    1
;
""").show()

+------------+--------------------+--------+
|PULocationID|                Zone|count(1)|
+------------+--------------------+--------+
|         105|Governor's Island...|       1|
+------------+--------------------+--------+

